# 13.4. p2wpkh와 p2sh-p2wpkh 코딩하기

In [1]:
class Tx:
    command = b'tx'

    def __init__(self, version, tx_ins, tx_outs,locktime, testnet=False, segwit = False):
        self.version = version
        self.tx_ins = tx_ins
        self.tx_outs = tx_outs
        self.locktime = locktime
        self.testnet = testnet
        self.segwit = segwit
        self._hash_prevouts = None
        self._hash_sequence = None
        self._hash_outputs=None
        

In [3]:
class Tx:
    @classmethod
    def parse(cls,s,testnet=False):
        s.read(4) # <1> 세그윗 트랜잭션인지 확인하기 위해 다섯 번쨰 바이트를 조사한다.
        if s.read(1)==b'\x00': # <2> : 다섯 번쨰 바이트가 0이면 세그윗 트랜잭션이다.
            parse_method = cls.parse_segwit
        else:
            parse_method = cls.parse_legacy
        s.seek(-5,1) # <3> : 스트림 포인터를 처음 5바이트를 조사하기 전으로 돌려놓는다.
        return parse_method(s,testnet=testnet)

    @classmethod
    def parse_legacy(cls,s,testnet=False):
        version = little_endian_to_int(s.read(4)) # <4> : 예전 parse 메서드의 이름을 parse_legacy로 변경한다.
        num_inputs = read_varint(s)
        inputs=[]
        for _ in range(num_outputs):
            inputs.append(TxIn.parse(s))
        num_outputs=read_varint(s)
        outputs=[]
        for _ in range(num_outputs):
            outputs.append(TxOut.parse(s))
        locktime = little_endian_to_int(s.read(4))
        return cls(version, inputs, outputs, locktime, testnet =testnet, segwit=False)
        

In [4]:
Tx.__init__

<slot wrapper '__init__' of 'object' objects>

In [5]:
Tx.parse

<bound method Tx.parse of <class '__main__.Tx'>>

In [6]:
Tx.parse_legacy

<bound method Tx.parse_legacy of <class '__main__.Tx'>>

In [7]:
class Tx:
    @classmethod
    def parse_segwit(cls, s, testnet=False):
        version = little_endian_to_int(s.read(4))
        marker = s.read(2)
        if marker != b'\x00\x01':  # <1> 2개의 새로운 필드가 있는데 그 중 하나가 세그윗 마커이다.
            raise RuntimeError('Not a segwit transaction {}'.format(marker))
        num_inputs = read_varint(s)
        inputs = []
        for _ in range(num_inputs):
            inputs.append(TxIn.parse(s))
        num_outputs = read_varint(s)
        outputs = []
        for _ in range(num_outputs):
            outputs.append(TxOut.parse(s))
        for tx_in in inputs:  # <2>
            num_items = read_varint(s)
            items = []
            for _ in range(num_items):
                item_len = read_varint(s)
                if item_len == 0:
                    items.append(0)
                else:
                    items.append(s.read(item_len))
            tx_in.witness = items
        locktime = little_endian_to_int(s.read(4))
        return cls(version, inputs, outputs, locktime, 
                   testnet=testnet, segwit=True)

In [8]:
class Tx:
    @classmethod
    def serialize(self):
        if self.segwit:
            return self.serialize_segwit()
        else:
            return self.serialize_legacy()

    def serialize_legacy(self):  # <1> 예전 serialize 메서드의 이름을 변경한다.
        result = int_to_little_endian(self.version, 4)
        result += encode_varint(len(self.tx_ins))
        for tx_in in self.tx_ins:
            result += tx_in.serialize()
        result += encode_varint(len(self.tx_outs))
        for tx_out in self.tx_outs:
            result += tx_out.serialize()
        result += int_to_little_endian(self.locktime, 4)
        return result

    def serialize_segwit(self):
        result = int_to_little_endian(self.version, 4)
        result += b'\x00\x01'  # <2> 세그윗 직렬화에서 마커를 추가한다.
        result += encode_varint(len(self.tx_ins))
        for tx_in in self.tx_ins:
            result += tx_in.serialize()
        result += encode_varint(len(self.tx_outs))
        for tx_out in self.tx_outs:
            result += tx_out.serialize()
        for tx_in in self.tx_ins:  # <3> 증인필드는 마지막에 직렬화된다.
            result += int_to_little_endian(len(tx_in.witness), 1)
            for item in tx_in.witness:
                if type(item) == int:
                    result += int_to_little_endian(item, 1)
                else:
                    result += encode_varint(len(item)) + item
        result += int_to_little_endian(self.locktime, 4)
        return result

In [9]:
def verify_input(self, input_index):
        '''Returns whether the input has a valid signature'''
        tx_in = self.tx_ins[input_index]
        script_pubkey = tx_in.script_pubkey(testnet=self.testnet)
        if script_pubkey.is_p2sh_script_pubkey():
            cmd = tx_in.script_sig.cmds[-1]
            raw_redeem = int_to_little_endian(len(cmd), 1) + cmd
            redeem_script = Script.parse(BytesIO(raw_redeem))
            if redeem_script.is_p2wpkh_script_pubkey(): # <1> 만약 if 문의 조건이 참이면 스크립트인 경우이다.
                z = self.sig_hash_bip143(input_index, redeem_script) # <2> 서명해시 값 계산함수는 정의되어 있다.
                witness = tx_in.witness
            elif redeem_script.is_p2wsh_script_pubkey():
                cmd = tx_in.witness[-1]
                raw_witness = encode_varint(len(cmd)) + cmd
                witness_script = Script.parse(BytesIO(raw_witness))
                z = self.sig_hash_bip143(input_index, witness_script=witness_script)
                witness = tx_in.witness
            else:
                z = self.sig_hash(input_index, redeem_script)
                witness = None
        else:
            if script_pubkey.is_p2wpkh_script_pubkey(): # <3> 만약 if문이 참이면 스크립트의 경우이다.
                z = self.sig_hash_bip143(input_index)
                witness = tx_in.witness
            elif script_pubkey.is_p2wsh_script_pubkey():
                cmd = tx_in.witness[-1]
                raw_witness = encode_varint(len(cmd)) + cmd
                witness_script = Script.parse(BytesIO(raw_witness))
                z = self.sig_hash_bip143(input_index, witness_script=witness_script)
                witness = tx_in.witness
            else:
                z = self.sig_hash(input_index)
                witness = None
        combined = tx_in.script_sig + script_pubkey
        return combined.evaluate(z, witness) # <4> witness를 스크립트 실행 메서드에 인수로 넘긴다.

In [10]:
class Tx:
    def verify_input(self, input_index):
        '''Returns whether the input has a valid signature'''
        tx_in = self.tx_ins[input_index]
        script_pubkey = tx_in.script_pubkey(testnet=self.testnet)
        if script_pubkey.is_p2sh_script_pubkey():
            cmd = tx_in.script_sig.cmds[-1]
            raw_redeem = int_to_little_endian(len(cmd), 1) + cmd
            redeem_script = Script.parse(BytesIO(raw_redeem))
            if redeem_script.is_p2wpkh_script_pubkey():
                z = self.sig_hash_bip143(input_index, redeem_script)
                witness = tx_in.witness
            elif redeem_script.is_p2wsh_script_pubkey(): # <1> p2sh-p2wsh의 경우인지 판단한다.
                cmd = tx_in.witness[-1]
                raw_witness = encode_varint(len(cmd)) + cmd
                witness_script = Script.parse(BytesIO(raw_witness))
                z = self.sig_hash_bip143(input_index, witness_script=witness_script)
                witness = tx_in.witness
            else:
                z = self.sig_hash(input_index, redeem_script)
                witness = None
        else:
            if script_pubkey.is_p2wpkh_script_pubkey():
                z = self.sig_hash_bip143(input_index)
                witness = tx_in.witness
            elif script_pubkey.is_p2wsh_script_pubkey(): # <2> p2wsh의 경우인지 판단한다.
                cmd = tx_in.witness[-1]
                raw_witness = encode_varint(len(cmd)) + cmd
                witness_script = Script.parse(BytesIO(raw_witness))
                z = self.sig_hash_bip143(input_index, witness_script=witness_script)
                witness = tx_in.witness
            else:
                z = self.sig_hash(input_index)
                witness = None
        combined = tx_in.script_sig + script_pubkey
        return combined.evaluate(z, witness)

__main__.Tx